# 1.02 Mixed Layer Crossing Locations

---

Author: Riley X. Brady

Date: 11/18/2020

This notebook pulls in the 19,002 particles that have been identified as those who last cross 1000 m in the ACC (S of 45S and outside of the annual sea ice zone) and then finds the x/y position in which they first cross 200 m following that deep upwelling. These crossing locations are further filtered down into those that cross into 200 m S of 45S and outside of the 75% annual sea ice zone. (N = 12,301)

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr

from dask.distributed import Client

In [2]:
print(f"xarray: {xr.__version__}")
print(f"numpy: {np.__version__}")

xarray: 0.16.1
numpy: 1.19.4


In [3]:
# This is my TCP client from the `launch_cluster` notebook. I use it
# for distributed computing with `dask` on NCAR's machine, Casper.
client = Client("tcp://...")

**Note**: I loaded in the netCDF file, and chunked it, and then saved it back out as a `zarr` file. This makes `dask` run a lot more efficiently. E.g.,

```python
ds = xr.open_dataset('../data/southern_ocean_deep_upwelling_particles.nc')
ds = ds.chunk({'time': -1, 'nParticles': 'auto'})
ds.to_zarr('../data/southern_ocean_deep_upwelling_particles.zarr', consolidated=True)
```

You could probably chunk the particles into slightly smaller chunks for even faster performance.

In [4]:
# Load in the `zarr` file, which is pre-chunked and already has been
# filtered from the original 1,000,000 particles to the 19,002 that
# upwell last across 1000 m S of 45S and outside of the annual sea ice
# edge.
filepath = "../data/southern_ocean_deep_upwelling_particles.zarr/"
ds = xr.open_zarr(filepath, consolidated=True)

In [5]:
def _compute_idx_of_first_200m_crossing(z):
    """Find first time particle upwells across 200 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999
    cond = (currentDepth >= -200) & (previousDepth < -200)
    idx = cond.argmax()
    return idx


def _compute_idx_of_last_1000m_crossing(z):
    """Find index of final time particle upwells across 1000 m.

    z : zLevelParticle
    """
    currentDepth = z
    previousDepth = np.roll(z, 1)
    previousDepth[0] = 999  # So we're not dealing with a nan here.
    cond = (currentDepth >= -1000) & (previousDepth < -1000)
    idx = (
        len(cond) - np.flip(cond).argmax() - 1
    )  # Finds last location that condition is true.
    return idx


def compute_xy_of_mixed_layer_crossing(x, y, z):
    """Find the x, y location of the mixed layer crossing.

    First compute the last time it crosses 1000 m, then subset
    to time series following that and find its first 200 m
    crossing.

    x : lonParticle (radians)
    y : latParticle (radians)
    z : zLevelParticle (m)
    """
    idx_a = _compute_idx_of_last_1000m_crossing(z)
    x_subset = x[idx_a - 1 : :]
    y_subset = y[idx_a - 1 : :]
    z_subset = z[idx_a - 1 : :]
    idx_b = _compute_idx_of_first_200m_crossing(z_subset)
    lon = np.rad2deg(x_subset[idx_b])
    lat = np.rad2deg(y_subset[idx_b])
    return np.array([lon, lat])

This applies the above functions to find the x/y locations of where the particles first cross 200 m following their last 1000 m crossing.

In [6]:
result = xr.apply_ufunc(
    compute_xy_of_mixed_layer_crossing,
    ds.lonParticle,
    ds.latParticle,
    ds.zLevelParticle,
    input_core_dims=[["time"], ["time"], ["time"]],
    vectorize=True,
    dask="parallelized",
    output_core_dims=[["coordinate"]],
    output_dtypes=[float],
    dask_gufunc_kwargs={"output_sizes": {"coordinate": 2}},
)

In [7]:
%time crossings = result.compute()

CPU times: user 9.27 ms, sys: 1.28 ms, total: 10.5 ms
Wall time: 1.4 s


In [8]:
crossings = crossings.assign_coords(coordinate=["x", "y"])

## Filter

---

Now we filter to our designations for a particle that upwells within the ACC. The original ensemble of 19,002 particles here was already filtered to meet those specifications for the *1000 m upwelling*, but now we need to re-apply them for the 200 m upwelling.

1. Upwell S of 45S.

In [9]:
crossings = crossings.where(crossings.sel(coordinate="y") < -45, drop=True)

2. Upwell outside of the 75% sea ice present annual climatology.

In [10]:
# Load in sea ice climatology from Eulerian mesh.
sea_ice = xr.open_dataset("../data/eulerian_sea_ice_climatology.nc")["icePresent"].mean(
    "month"
)

# Load in mesh info to get lat/lon of grid cells.
mesh_info = xr.open_dataset("../data/mesh.nc")
mpas_lat = np.rad2deg(mesh_info.latCell)
mpas_lon = np.rad2deg(mesh_info.lonCell)

# To save some computational cost, only look S of 45S.
sea_ice = sea_ice.where(mpas_lat <= -45, drop=True)
mpas_lon = mpas_lon.where(mpas_lat <= -45, drop=True)
mpas_lat = mpas_lat.where(mpas_lat <= -45, drop=True)

In [11]:
def _find_mpas_cell(mpas_lon, mpas_lat, xParticle, yParticle):
    """Returns the idx of the closest mpas cell to the particle
    location.

    mpas_lon : Longitude of Eulerian mesh in degrees.
    mpas_lat : Latitude of Eulerian mesh in degrees.
    xParticle : lonParticle in degrees.
    yParticle : latParticle in degrees.
    """
    dx = mpas_lon - xParticle
    dy = mpas_lat - yParticle
    diff = abs(dx) + abs(dy)
    idx = np.nanargmin(diff)
    return idx

In [12]:
# Chunk particle crossings and MPAS mesh information to make
# processing easier.
xCross = crossings.sel(coordinate="x").chunk({"nParticles": 5000}).persist()
yCross = crossings.sel(coordinate="y").chunk({"nParticles": 5000}).persist()
mpas_lon = mpas_lon.chunk().persist()
mpas_lat = mpas_lat.chunk().persist()

In [13]:
result = xr.apply_ufunc(
    _find_mpas_cell,
    mpas_lon,
    mpas_lat,
    xCross,
    yCross,
    input_core_dims=[["nCells"], ["nCells"], [], []],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[float],
)

In [14]:
# returns the index on the mpas sea ice mesh of where
# the particle crossing occurred.
%time result = result.compute()

CPU times: user 21.5 ms, sys: 2.44 ms, total: 24 ms
Wall time: 19.2 s


Creates a mask for particles that upwell outside of the 75% sea ice present zone.

In [15]:
outside_of_ice_mask = (sea_ice < 0.75).isel(nCells=result.astype("int"))
crossings = crossings.where(outside_of_ice_mask, drop=True)

In [16]:
ds = xr.Dataset()
ds["lon_crossing"] = crossings.sel(coordinate="x")
ds["lat_crossing"] = crossings.sel(coordinate="y")
ds.attrs[
    "description"
] = "x/y locations of first 200m crossing following the *final* 1000m crossing for particles that occur < 45S; reach 200 m following this crossing; and happen outside of the 75% annual sea ice zone. The S of 45S and sea ice mask was re-applied to the 200m crossings."
ds.to_netcdf("../data/postproc/200m.crossing.locations.nc")